In [1]:

# data manipulation
import pandas as pd

# plotly 
import plotly.express as px
import plotly.graph_objects as go
# dashboards
import dash 
from jupyter_dash import JupyterDash #for running the dashboard in jupyter notebook
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input
import dash_bootstrap_components as dbc #for dashboard themes

import numpy as np
import warnings
warnings.filterwarnings('ignore')


<ipython-input-1-1f663c3dc0e1>:10: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-1-1f663c3dc0e1>:11: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [41]:
def calc_cd(data,df,team_value):
    d=data[data['team']==team_value].sort_values(by='datetime', key=pd.to_datetime)
    for season_value in d['season'].unique():
        dd=d[d['season']==season_value]
        dd['cd']=np.cumsum(dd['distance_traveled'])
        for row in dd.index:
            df.loc[row]=dd.loc[row]
            df.loc[row,'cd']=dd.loc[row,'cd']
    return df

In [104]:
data = pd.read_csv('schedule.csv')
l=list(data.columns)
l.append('cd')
l.append('acd')
df=pd.DataFrame(columns=l, index=range(0,data.shape[0]))
for team_value in data['team'].unique():
    df=calc_cd(data,df,team_value)
df.to_csv('schedule_cd.csv')
writer.save()
df.team.unique()

array(['POR', 'LAC', 'DEN', 'HOU', 'DAL', 'NOH', 'DET', 'CHI', 'PHO',
       'SAC', 'NJN', 'TOR', 'IND', 'PHI', 'WAS', 'MIL', 'MIN', 'NYK',
       'MIA', 'ORL', 'MEM', 'BOS', 'GSW', 'UTA', 'SAS', 'CLE', 'CHA',
       'OKC', 'ATL', 'LAL', 'BKN', 'NOP', 'CHO'], dtype=object)

In [114]:
c=df.groupby('datetime')['cd','season'].apply(np.mean)
ix=c.index.sort_values(key=pd.to_datetime)
df2=pd.DataFrame(columns=['season','acd'],index=ix)
for d in ix:
    for i in df[df.datetime==d].index:
        df2.loc[d,'season'] = c.loc[d,'season']
        df2.loc[d,'acd']=c.loc[d,'cd']
df2.to_csv('schedule_acd.csv')
writer.save()
df2

,season,acd
datetime,,
10/28/08,2009.0,371.585011
10/29/08,2009.0,724.184535
10/30/08,2009.0,1279.3232
10/31/08,2009.0,1271.261259
11/1/08,2009.0,1612.040254
...,...,...
4/6/22,2022.0,42427.960581
4/7/22,2022.0,43317.72246
4/8/22,2022.0,41727.799754


In [130]:
fig = go.Figure()
df2 = pd.read_csv('schedule_acd.csv',index_col=0)
df2=df2[df2['season']==2022]

fig.add_trace(
    go.Scatter(x = df2.index, 
            y = df2['acd'],
            mode = 'lines',
            line={'color': 'green'},
            name='Average',
            )
)
fig.show()

In [133]:
team_value='ATL'
data = pd.read_csv('schedule_cd.csv')
d=data[data['team']==team_value].sort_values(by='datetime', key=pd.to_datetime)
d=d[d['season']==2022]

fig = go.Figure()
fig.add_trace(
    go.Scatter(x = d['datetime'], 
            y = d['cd'],
            mode = 'lines',
            line={'color': 'green'},
            name=team_value,
            )
)
fig.show()

In [135]:
app = JupyterDash(__name__,external_stylesheets = [dbc.themes.BOOTSTRAP])

# read the data
data = pd.read_csv('schedule.csv')

# list of all unique teams
teams=data['opponent_name'].unique()

# list of all unique team abbreviation
team_abbr = data['opponent_abbr'].unique()

#list of all unique seasons
seasons=data['season'].unique()

app.layout = html.Div([
    
    # top left drop menu for teams
    html.Div([
            dcc.Dropdown(
                id='team_value',
                options=[{'label': teams[i], 'value': team_abbr[i]} for i in range(len(teams))],
                value='ATL'
            )
    ], style={'width': '30%', 'display': 'inline-block'}), 
   
    
    # top middle drop menu for seasons
    html.Div([
            dcc.Dropdown(
                id='season_value',
                options=[{'label': s, 'value': s} for s in seasons],
                value=2022
            )
    ], style={'width': '30%', 'float': 'center', 'display': 'inline-block'}),


    # top left date range picker 
    html.Div([
        dcc.DatePickerRange(
            id='datepicker',
            month_format='YYYY MMM'
        )
    ], style={'width': '30%', 'float': 'right', 'display': 'inline-block'}),

    dcc.Graph(id='cumdist-graphic')
])
@app.callback(
    Output('datepicker', 'start_date'),
    Output('datepicker', 'end_date'),
    Input('season_value', 'value')
)

def update_date_range(season_value): 
    df = pd.read_csv('schedule.csv', index_col=0)
    df = df[df['season']==season_value]
    df = df[df['playoffs']==0]
    min_date = df['datetime'].min()
    max_date = df['datetime'].max()
    return min_date, max_date

@app.callback(
    Output('cumdist-graphic', 'figure'),
    Input('team_value', 'value'),
    Input('season_value', 'value'),
    Input('datepicker', 'start_date'),
    Input('datepicker', 'end_date'))

def update_graph(team_value, season_value, start_date, end_date):
    
    fig = go.Figure()

    df2 = pd.read_csv('schedule_acd.csv',index_col=0)
    df2 = df2[df2['season']==season_value]
    df = df2[(df2.index > start_date) & (df2.index < end_date)]

    df = pd.read_csv('schedule_cd.csv')

    df=df[df['season']==season_value]
    df = df[(df['datetime'] > start_date) & (df['datetime'] < end_date)]

    d=df[df['team']==team_value].sort_values(by='datetime', key=pd.to_datetime)

    fig.add_trace(
        go.Scatter(x = d['datetime'], 
                y = df2['acd'],
                mode = 'lines',
                line={'color': 'grey'},
                name='Average',
                )
    )

    fig.add_trace(
        go.Scatter(x = d['datetime'], 
                y = d['cd'],
                mode = 'lines',
                line={'color': 'green'},
                name=team_value,
                )
    )
    
    return fig
    
if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
